Name: Voronova Aleksandra

Group: BS21-DS-02

# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


# Original Norvig's solution

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
import re
from collections import Counter

def Norvig_words(text):
    return re.findall(r'\w+', text.lower())

WORDS = Counter(Norvig_words(open('/content/drive/MyDrive/nlp-assignment-2/big.txt').read()))

def Norvig_P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    return WORDS[word] / N

def Norvig_correction(word):
    "Most probable spelling correction for word."
    return max(Norvig_candidates(word), key=Norvig_P)

def Norvig_candidates(word):
    "Generate possible spelling corrections for word."
    return (Norvig_known([word]) or Norvig_known(Norvig_edits1(word)) or Norvig_known(Norvig_edits2(word)) or [word])

def Norvig_known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def Norvig_edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def Norvig_edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in Norvig_edits1(word) for e2 in Norvig_edits1(e1))

In [3]:
Norvig_correction('speling')

'spelling'

## Implement context-sensitive spelling correction

Your task is to **implement** context-sensitive spelling corrector using N-gram language model. The idea is to **compute conditional probabilities** of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- **efficiency improvement** to make the solution faster,
- any other idea of yours to **improve** the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- **Your** implementation
- **Analysis** of why the implemented approach is suggested
- **Improvements** of the original approach that you have chosen to implement

In [4]:
import pandas as pd
import numpy as np

bigram_df = pd.read_csv('/content/drive/MyDrive/nlp-assignment-2/bigrams.txt',
                        delimiter='\t', names=['frequency', 'context', 'word'], header=None)
bigram_df

,frequency,context,word
0,275,a,a
1,31,a,aaa
2,29,a,all
3,45,a,an
4,192,a,and
...,...,...,...
1020380,24,zviad,gamsakhurdia
1020381,25,zweimal,leben
1020382,24,zwick,and
1020383,24,zydeco,music


In [19]:
import nltk
import warnings
import math

Ngram_WORDS = set(list(bigram_df['word'].unique()))

In [20]:
N_bigram_words = bigram_df['frequency'].sum()
bigram_matrix = bigram_df.to_numpy()

def Ngram_P(word, N=N_bigram_words):
    "Probability of `word`."
    try:
        return math.log(np.sum(bigram_matrix[np.where(bigram_matrix[:, 2] == word)][:, 0]) / N)
    except Exception as e:
        return 0

def Ngram_correction(context, word):
    "Most probable spelling correction for word."
    context_sum = np.sum(bigram_matrix[np.where(bigram_matrix[:, 1] == context)][:, 0])
    candidates = Ngram_candidates(word)
    candidates_df = bigram_matrix[np.where((bigram_matrix[:, 1] == context) & (np.isin(bigram_matrix[:, 2], candidates)))].copy()
    candidates_df[:, 0] = [math.log(freq / context_sum) for freq in candidates_df[:, 0]]
    candidates_df[:, 0] += [Ngram_P(cnd) for cnd in candidates_df[:, 2]]
    candidates_df = candidates_df[np.argsort(candidates_df[:, 0])[::-1]]

    if len(candidates_df) == 0:
        return max(candidates, key=Ngram_P)

    return candidates_df[0, 2]

def Ngram_candidates(word):
    "Generate possible spelling corrections for word."
    return (Ngram_known([word]) or Ngram_known(Ngram_edits1(word)) or Ngram_known(Ngram_edits2(word)) or [word])

def Ngram_known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in Ngram_WORDS)

def Ngram_edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def Ngram_edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in Ngram_edits1(word) for e2 in Ngram_edits1(e1))

In [21]:
Ngram_correction('the', 'gren')

'green'

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

## Answer

First of all, it's important to talk about the pros and cons of Norvig's solution.

Since Norvig's solution works within a single word, looking at its possible typos, this makes the solution fast, simple and, judging by the metrics, quite accurate.

However, because Norvig's solution does not take context into account, the probability of proper correction is reduced.

My implementation is a modification of Norvig's solution, where the probability calculation takes into account not only the frequency of the word's occurrence in the dictionary, but also the occurrence of the word in the context, making the choice of the appropriate word more accurate.


**Assumptions:**
* Works only in context
* Depends on the dictionary size

Perhaps my modification doesn't work in favor of the speed of the algorithm (since more calculations are added), but the accuracy obviously increases due to the consideration of context. And this is an undeniable advantage of the N-gram.

There is no point in using trigrams and larger, since it will unnecessarily overload the solution. However, bigrams can always be extended by adding more words and context to the dataset - this will increase the accuracy of the model much more effectively.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [22]:
import random

test_df = pd.read_csv('/content/drive/MyDrive/nlp-assignment-2/test_sentences.txt',
                        delimiter='\t', names=['correct_sentences'], header=None)
test_df['correct_sentences'] = [text.replace(',', "")[:-1].lower() for text in test_df['correct_sentences']]
test_df

,correct_sentences
0,the quick brown fox jumps over the lazy dog
1,tomorrow will be a better day
2,she sells seashells by the seashore
3,he who hesitates is lost
4,a journey of a thousand miles begins with a si...
...,...
95,haste makes waste
96,if at first you don't succeed try try again
97,if the shoe fits wear it
98,if you can't beat them join them


In [23]:
import string
import random

def test_edit1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def test_edit2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in test_edit1(word) for e2 in test_edit1(e1))

def test_gen_wrong_sentence(text):
    new_text = []
    for word in text.split():
        r = random.uniform(0, 1)
        new_text.append(random.choice(list(test_edit1(word)) + list(test_edit2(word))))
        if r < 0.05:
            new_text.append(word)
    new_text[0] = text.split()[0]
    return " ".join(new_text)

test_df['wrong_sentences'] = [test_gen_wrong_sentence(text) for text in test_df['correct_sentences']]
test_df

,correct_sentences,wrong_sentences
0,the quick brown fox jumps over the lazy dog,the quivbck brain fli hjumts oere tjle laeq sdor
1,tomorrow will be a better day,tomorrow nwisll bvt ayj bettenrx udya
2,she sells seashells by the seashore,she esellls seasheyils my krthe dseashorbe
3,he who hesitates is lost,he ugwho hdsitatys xes yldost
4,a journey of a thousand miles begins with a si...,a uourxney ojk dy txhousanw minvles begipnsz w...
...,...,...
95,haste makes waste,haste gmaqkes wiste
96,if at first you don't succeed try try again,if catk fimsmt aoj donfd psuicceed trhz tbrty ...
97,if the shoe fits wear it,if mthxe sohor fijns mtear itw
98,if you can't beat them join them,if yeow cman'ut beakx tthwem jotn thcem


In [26]:
import re

def correct_text(text, method):
    text = re.sub(r"[,?!.]", "", text)
    text = text.lower().split()
    for i in range(1, len(text)):
        if method == 'Ngram':
            text[i] = Ngram_correction(text[i-1], text[i])
        elif method == 'Norvig':
            text[i] = Norvig_correction(text[i])
    return " ".join(text)

test_df['Ngram_correction'] = [correct_text(text, method='Ngram') for text in test_df['wrong_sentences']]
test_df['Norvig_correction'] = [correct_text(text, method='Norvig') for text in test_df['wrong_sentences']]

In [27]:
test_df

,correct_sentences,wrong_sentences,Ngram_correction,Norvig_correction
0,the quick brown fox jumps over the lazy dog,the quivbck brain fli hjumts oere tjle laeq sdor,the quick brain fly hurts were tale lael odor,the quick brain fly hurts were tale last odor
1,tomorrow will be a better day,tomorrow nwisll bvt ayj bettenrx udya,tomorrow will but aye better day,tomorrow will but ay better day
2,she sells seashells by the seashore,she esellls seasheyils my krthe dseashorbe,she sells seashells my the seashore,she esellls seasheyils my the seashore
3,he who hesitates is lost,he ugwho hdsitatys xes yldost,he who hesitates yes almost,he who hdsitatys yes almost
4,a journey of a thousand miles begins with a si...,a uourxney ojk dy txhousanw minvles begipnsz w...,a journey ok by thousand minutes begins width ...,a journey oak dy thousand minutes begins width...
...,...,...,...,...
95,haste makes waste,haste gmaqkes wiste,haste makes waste,haste makes wise
96,if at first you don't succeed try try again,if catk fimsmt aoj donfd psuicceed trhz tbrty ...,if cat first aol done succeed the try again,if cat first adj don succeed the party again
97,if the shoe fits wear it,if mthxe sohor fijns mtear itw,if the soho fins tear it,if the door finns tear it
98,if you can't beat them join them,if yeow cman'ut beakx tthwem jotn thcem,if yew can't beak them john them,if you cman'ut beak them john them


In [28]:
from tabulate import tabulate
from bert_score import score

Ngram_predictions = list(test_df['Ngram_correction'])
Norvig_predictions = list(test_df['Norvig_correction'])
references = list(test_df['correct_sentences'])

Ngram_P, Ngram_R, Ngram_F1 = score(Ngram_predictions, references, lang='en', verbose=False)
Norvig_P, Norvig_R, Norvig_F1 = score(Norvig_predictions, references, lang='en', verbose=False)

print(tabulate([['F1 score', Norvig_F1.mean(), Ngram_F1.mean()],
                ['Recall score', Norvig_R.mean(), Ngram_R.mean()],
                ['Precision score', Norvig_P.mean(), Ngram_P.mean()]],
               headers=['Norvig', 'Ngram'], tablefmt="grid", showindex=False))

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


+-----------------+----------+----------+
|                 |   Norvig |    Ngram |
+=================+==========+==========+
| F1 score        | 0.870495 | 0.877718 |
+-----------------+----------+----------+
| Recall score    | 0.874586 | 0.882313 |
+-----------------+----------+----------+
| Precision score | 0.866707 | 0.873374 |
+-----------------+----------+----------+


## Test dataframe generation
To generate the test dataframe, I used a ready-made set of sentences, which I then modified using the *test_edit1* and *test_edit2* functions.

## Evaluation
For evaluation I used BERTScore; powerful and easy-to-use pre-trained transformer. I chosed it since it's more nuanced and accurate in comparison of another traditional metrics.

## Comparison
As you can see on the cell above, N-gram solution definitely works more accurate than Norvig's solution, since N-gram takes context into account.